In [1]:
# 기초 전처리
import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm import tqdm


# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import os

# 캔버스 사이즈 적용
plt.rcParams['figure.figsize'] = (12, 9)

# 컬럼 전체 확인 가능하도록 출력 범위 설정
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

pd.reset_option('display.float_format')

# 파일 로드위한 directory 확인 및 현재 경로로 설정
a = os.getcwd()
os.chdir(a)

C:\Users\HB\anaconda3\envs\my_env\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# 좌표값 포함한 상권 영역 데이터 불러오기
bs_area = gpd.read_file('상권_geometry_좌표.geojson', driver= 'GeoJSON')

In [3]:
bs_area.head()

,TRDAR_SE_C,TRDAR_SE_1,TRDAR_CD,TRDAR_CD_N,XCNTS_VALU,YDNTS_VALU,SIGNGU_CD,ADSTRD_CD,STDR_YM_CD,area,perimeter,x,y,geometry
0,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,201810,15720.885857,614.725259,126.956149,37.478185,"MULTIPOLYGON (((126.95719 37.47880, 126.95689 ..."
1,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,201810,9643.280304,478.077580,126.945127,37.481142,"MULTIPOLYGON (((126.94621 37.48127, 126.94614 ..."
2,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,201810,18762.968506,819.374779,127.051464,37.497494,"MULTIPOLYGON (((127.05226 37.49757, 127.05251 ..."
3,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,201810,6289.532303,395.342913,127.068578,37.489217,"MULTIPOLYGON (((127.06765 37.48935, 127.06904 ..."
4,R,전통시장,1001412,화곡본동시장,186203,449328,11500,11500590,201810,21476.355774,584.875003,126.843882,37.543360,"MULTIPOLYGON (((126.84412 37.54273, 126.84379 ..."


In [4]:
# 이어 붙일 기본 파일
df_base = bs_area[['TRDAR_CD', 'TRDAR_SE_1']].sort_values('TRDAR_CD').reset_index(drop=True)
df_base.columns = ['상권_코드','상권_이름']
df_base['상권_코드'] = df_base['상권_코드'].apply(lambda x : str(x))
df_base.astype( {'상권_코드':object} )
# 지리적으로 중복되는 상권 영역인 관광특구 제외
not_bs_area = df_base['상권_이름'] !='관광특구'

df_base = df_base[not_bs_area]
df_base

,상권_코드,상권_이름
0,1000001,골목상권
1,1000002,골목상권
2,1000003,골목상권
3,1000004,골목상권
4,1000005,골목상권
...,...,...
1485,1001486,전통시장
1486,1001487,전통시장
1487,1001488,전통시장
1488,1001489,전통시장


In [5]:
# 추후 좌표 위치 처리용 기본 파일
df_gid = bs_area[['TRDAR_CD', 'TRDAR_SE_1', 'x', 'y', 'geometry']].sort_values('TRDAR_CD').reset_index(drop=True)
df_gid.columns = ['상권_코드','상권_이름', 'x', 'y', 'geometry']
df_gid['상권_코드'] = df_gid['상권_코드'].apply(lambda x : str(x))
df_gid.astype( {'상권_코드':object} )

# 지리적으로 중복되는 상권 영역인 관광특구 제외
not_bs_area2 = df_gid['상권_이름'] !='관광특구'

df_gid = df_gid[not_bs_area2]
df_gid

,상권_코드,상권_이름,x,y,geometry
0,1000001,골목상권,126.986400,37.582041,"MULTIPOLYGON (((126.98591 37.58549, 126.98626 ..."
1,1000002,골목상권,127.022589,37.573212,"MULTIPOLYGON (((127.02335 37.57240, 127.02335 ..."
2,1000003,골목상권,126.988420,37.575664,"MULTIPOLYGON (((126.98909 37.57418, 126.98771 ..."
3,1000004,골목상권,126.994996,37.588721,"MULTIPOLYGON (((126.99403 37.59000, 126.99419 ..."
4,1000005,골목상권,126.967764,37.597109,"MULTIPOLYGON (((126.97014 37.59995, 126.97301 ..."
...,...,...,...,...,...
1485,1001486,전통시장,127.128157,37.542089,"MULTIPOLYGON (((127.12908 37.54171, 127.12902 ..."
1486,1001487,전통시장,127.127667,37.541139,"MULTIPOLYGON (((127.12797 37.54047, 127.12794 ..."
1487,1001488,전통시장,127.127584,37.533863,"MULTIPOLYGON (((127.12847 37.53489, 127.12850 ..."
1488,1001489,전통시장,127.135463,37.527724,"MULTIPOLYGON (((127.13638 37.52813, 127.13629 ..."


# 서울시 전체 지하철  찾기


In [6]:
info_subway = pd.read_csv('서울시 지하철역 정보 검색 (역명).csv', encoding = 'cp949')

In [7]:
info_subway.head()

,전철역코드,전철역명,호선,외부코드
0,0159,동묘앞,01호선,127
1,0416,미아사거리,04호선,416
2,0419,한성대입구,04호선,419
3,0422,동대문역사문화공원,04호선,422
4,0423,충무로,04호선,423


## info_subway 전처리

In [8]:
# 끝이 '역' 으로 끝나지 않은 row에 '역' 을 붙여주기 
def func1(row):
    if row['전철역명'].endswith('역') == True:
        return row['전철역명']
    else:
        return row['전철역명'] +'역'
    
info_subway['전철역명'] = info_subway.apply(func1, axis=1)


# 각 호선에 숫자 0을 제거하기
def func2(row):
    if row['호선'].startswith('0') == True:
        return row['호선'][1:]
    else:
        return row['호선']
    
#호선 앞에 0 잘라내기
info_subway['호선'] = info_subway.apply(func2, axis=1)


# 지하철역 컬럼으로 이름 합치기
# 카카오 API와 동일한 형태로 변경 (e.g : '서울역 1호선')
info_subway['지하철역'] = info_subway['전철역명']+' '+info_subway['호선']

# 결과 확인
#info_subway = info_subway[info_subway['전철역명'].str.endswith('역')].reset_index(drop=True)

# 불필요 컬럼 제거
info_subway.drop(columns = ['전철역명','호선','외부코드'], inplace = True)

# 결과 확인
info_subway.head()

,전철역코드,지하철역
0,0159,동묘앞역 1호선
1,0416,미아사거리역 4호선
2,0419,한성대입구역 4호선
3,0422,동대문역사문화공원역 4호선
4,0423,충무로역 4호선


In [9]:
info_subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   전철역코드   748 non-null    object
 1   지하철역    748 non-null    object
dtypes: object(2)
memory usage: 11.8+ KB


## 카카오 API로 지하철 정보 찾기

In [10]:
import json
import requests

class KakaoLocalAPI:
    """
    Kakao Local API 컨트롤러
    """

    def __init__(self, rest_api_key):
        """
        Rest API키 초기화 및 기능 별 URL 설정
        """

        # REST API 키 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}

        # 서비스 별 URL 설정

        # 01 주소 검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        # 02 좌표-행정구역정보 변환
        self.URL_02 = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
        # 03 좌표-주소 변환
        self.URL_03 = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
        # 04 좌표계 변환
        self.URL_04 = "https://dapi.kakao.com/v2/local/geo/transcoord.json"
        # 05 키워드 검색
        self.URL_05 = "https://dapi.kakao.com/v2/local/search/keyword.json"
        # 06 카테고리 검색
        self.URL_06 = "https://dapi.kakao.com/v2/local/search/category.json"

   
    def search_keyword(self,query,category_group_code=None,x=None,y=None,radius=None,rect=None,page=None,size=None,sort=None):
        """
        05 키워드 검색
        """
        params = {"query": f"{query}"}
        
        if category_group_code != None:
            params['category_group_code'] = f"{category_group_code}"
        if x != None:
            params['x'] = f"{x}"
        if y != None:
            params['y'] = f"{y}"
        if radius != None:
            params['radius'] = f"{radius}"
        if rect != None:
            params['rect'] = f"{rect}"
        if page != None:
            params['page'] = f"{page}"
        if size != None:
            params['size'] = f"{params}"
        if sort != None:
            params['sort'] = f"{sort}"
        
        res = requests.get(self.URL_05, headers=self.headers, params=params)
        document = json.loads(res.text)
        
        return document

In [11]:
# REST API 키
rest_api_key = "개인 apikey"

kakao = KakaoLocalAPI(rest_api_key)

In [12]:
## Set params
query = info_subway["지하철역"]

## Request
result = []
for i in tqdm(query):
    result.append(kakao.search_keyword(i) )

100%|████████████████████████████████████████████████████████████████████████████████| 748/748 [01:37<00:00,  7.66it/s]


In [13]:
import pprint as pp
pp.pprint(result)

[{'documents': [{'address_name': '서울 종로구 숭인동 117',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 수도권1호선',
                 'distance': '',
                 'id': '21160608',
                 'phone': '02-6110-1271',
                 'place_name': '동묘앞역 1호선',
                 'place_url': 'http://place.map.kakao.com/21160608',
                 'road_address_name': '서울 종로구 종로 359',
                 'x': '127.016364227103',
                 'y': '37.5732166600283'}],
  'meta': {'is_end': True,
           'pageable_count': 1,
           'same_name': {'keyword': '동묘앞역 1호선',
                         'region': [],
                         'selected_region': ''},
           'total_count': 5}},
 {'documents': [{'address_name': '서울 강북구 미아동 66-1',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 수도권4호

                 'place_url': 'http://place.map.kakao.com/21160924',
                 'road_address_name': '서울 성동구 왕십리광장로 17',
                 'x': '127.0383329389645',
                 'y': '37.56205814456861'},
                {'address_name': '서울 성동구 옥수동 168-1',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 경의중앙선',
                 'distance': '',
                 'id': '21161054',
                 'phone': '1544-7788',
                 'place_name': '옥수역 경의중앙선',
                 'place_url': 'http://place.map.kakao.com/21161054',
                 'road_address_name': '서울 성동구 동호로 21',
                 'x': '127.018746821679',
                 'y': '37.5405137035097'}],
  'meta': {'is_end': True,
           'pageable_count': 3,
           'same_name': {'keyword': '경의선',
                         'region': [],
                         'selected_region': '서울 성동구 응봉역'},
           'total

           'total_count': 2}},
 {'documents': [{'address_name': '경기 구리시 인창동 244-2',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 경의중앙선',
                 'distance': '',
                 'id': '21160922',
                 'phone': '031-552-7784',
                 'place_name': '구리역 경의중앙선',
                 'place_url': 'http://place.map.kakao.com/21160922',
                 'road_address_name': '경기 구리시 건원대로34번길 32-29',
                 'x': '127.143350984187',
                 'y': '37.6032707405088'},
                {'address_name': '경기 남양주시 다산동 4056-1',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 경의중앙선',
                 'distance': '',
                 'id': '21160735',
                 'phone': '1544-7788',
                 'place_name': '도농역 경의중앙선',
                 'place_url':

                 'id': '21160583',
                 'phone': '02-6110-2421',
                 'place_name': '아현역 2호선',
                 'place_url': 'http://place.map.kakao.com/21160583',
                 'road_address_name': '서울 마포구 신촌로 지하 270',
                 'x': '126.95614644008904',
                 'y': '37.55740617797663'}],
  'meta': {'is_end': True,
           'pageable_count': 1,
           'same_name': {'keyword': '아현역 2호선',
                         'region': [],
                         'selected_region': ''},
           'total_count': 5}},
 {'documents': [{'address_name': '서울 은평구 진관동 67-26',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 수도권3호선',
                 'distance': '',
                 'id': '21160820',
                 'phone': '02-6110-3201',
                 'place_name': '구파발역 3호선',
                 'place_url': 'http://place.map.kakao.com/21160820',
        

                         'selected_region': ''},
           'total_count': 5}},
 {'documents': [{'address_name': '강원 춘천시 남산면 방곡리 409',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 경춘선',
                 'distance': '',
                 'id': '12706616',
                 'phone': '1544-7788',
                 'place_name': '강촌역 경춘선',
                 'place_url': 'http://place.map.kakao.com/12706616',
                 'road_address_name': '강원 춘천시 남산면 강촌로 150',
                 'x': '127.634151572422',
                 'y': '37.805721704842'},
                {'address_name': '강원 춘천시 남산면 강촌리 680-7',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 경춘선',
                 'distance': '',
                 'id': '8319016',
                 'phone': '033-262-8827',
                 'place_name': 

...

                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 경의중앙선',
                 'distance': '',
                 'id': '26352286',
                 'phone': '1544-7788',
                 'place_name': '일산역 경의중앙선',
                 'place_url': 'http://place.map.kakao.com/26352286',
                 'road_address_name': '경기 고양시 일산서구 경의로 672',
                 'x': '126.770022642636',
                 'y': '37.6820129415088'}],
  'meta': {'is_end': True,
           'pageable_count': 1,
           'same_name': {'keyword': '일산역 경의선',
                         'region': [],
                         'selected_region': ''},
           'total_count': 1}},
 {'documents': [{'address_name': '경기 고양시 일산서구 덕이동 238-12',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 경의중앙선',
                 'distance': '',
      

                 'place_name': '검암역 인천2호선',
                 'place_url': 'http://place.map.kakao.com/7399940',
                 'road_address_name': '인천 서구 검바위로 4',
                 'x': '126.67567426472',
                 'y': '37.5686781799536'},
                {'address_name': '인천 서구 검암동 370-25',
                 'category_group_code': 'PK6',
                 'category_group_name': '주차장',
                 'category_name': '교통,수송 > 교통시설 > 주차장',
                 'distance': '',
                 'id': '1376473995',
                 'phone': '1899-1485',
                 'place_name': 'AJ파크 인천2호선검암역점',
                 'place_url': 'http://place.map.kakao.com/1376473995',
                 'road_address_name': '인천 서구 검바위로 4',
                 'x': '126.675706177106',
                 'y': '37.5686278120734'}],
  'meta': {'is_end': True,
           'pageable_count': 7,
           'same_name': {'keyword': '인천2호선',
                         'region': [],
                         'selected_

                 'id': '338574240',
                 'phone': '02-3499-5561',
                 'place_name': '가오리역 우이신설선',
                 'place_url': 'http://place.map.kakao.com/338574240',
                 'road_address_name': '서울 강북구 삼양로 426',
                 'x': '127.01676896839822',
                 'y': '37.6415440572233'},
                {'address_name': '서울 강북구 미아동 791-4755',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 우이신설경전철',
                 'distance': '',
                 'id': '609233775',
                 'phone': '02-3499-5561',
                 'place_name': '삼양역 우이신설선',
                 'place_url': 'http://place.map.kakao.com/609233775',
                 'road_address_name': '서울 강북구 삼양로 지하 259',
                 'x': '127.018156780705',
                 'y': '37.6269190539283'},
                {'address_name': '서울 강북구 미아동 791-2244',
                 'category

                 'place_url': 'http://place.map.kakao.com/622214266',
                 'road_address_name': '서울 동대문구 회기로 196',
                 'x': '127.058048369273',
                 'y': '37.5897962196601'},
                {'address_name': '서울 종로구 창신동 492-1',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 수도권1호선',
                 'distance': '',
                 'id': '21160565',
                 'phone': '02-6110-1281',
                 'place_name': '동대문역 1호선',
                 'place_url': 'http://place.map.kakao.com/21160565',
                 'road_address_name': '서울 종로구 종로 지하 302',
                 'x': '127.01063381083677',
                 'y': '37.571669405802616'},
                {'address_name': '서울 금천구 가산동 717',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 수도권1호선',
   

                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 용인에버라인',
                 'distance': '',
                 'id': '19471020',
                 'phone': '031-329-3577',
                 'place_name': '고진역 에버라인',
                 'place_url': 'http://place.map.kakao.com/19471020',
                 'road_address_name': '경기 용인시 처인구 경안천로 141',
                 'x': '127.21418360142515',
                 'y': '37.244638336050315'}],
  'meta': {'is_end': True,
           'pageable_count': 3,
           'same_name': {'keyword': '용인경전철',
                         'region': [],
                         'selected_region': '경기 용인시 보평역'},
           'total_count': 3}},
 {'documents': [{'address_name': '경기 용인시 처인구 포곡읍 둔전리 406-93',
                 'category_group_code': 'SW8',
                 'category_group_name': '지하철역',
                 'category_name': '교통,수송 > 지하철,전철 > 용인에버라인',
                 'di

In [14]:
a = pd.DataFrame(result)
b = pd.DataFrame(a['documents'])

In [15]:
subway_word = pd.DataFrame()
for i in tqdm(range(0,len(b))):
    c = pd.DataFrame(b['documents'][i])
    subway_word = pd.concat( (subway_word, c), axis=0, ignore_index = True)

100%|███████████████████████████████████████████████████████████████████████████████| 748/748 [00:01<00:00, 559.28it/s]


In [16]:
subway_word.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1589 entries, 0 to 1588
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   address_name         1589 non-null   object
 1   category_group_code  1589 non-null   object
 2   category_group_name  1589 non-null   object
 3   category_name        1589 non-null   object
 4   distance             1589 non-null   object
 5   id                   1589 non-null   object
 6   phone                1589 non-null   object
 7   place_name           1589 non-null   object
 8   place_url            1589 non-null   object
 9   road_address_name    1589 non-null   object
 10  x                    1589 non-null   object
 11  y                    1589 non-null   object
dtypes: object(12)
memory usage: 149.1+ KB


In [17]:
# 지하철역만 선별
subway_word = subway_word[subway_word['category_group_name']=='지하철역']
subway_word.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1296 entries, 0 to 1588
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   address_name         1296 non-null   object
 1   category_group_code  1296 non-null   object
 2   category_group_name  1296 non-null   object
 3   category_name        1296 non-null   object
 4   distance             1296 non-null   object
 5   id                   1296 non-null   object
 6   phone                1296 non-null   object
 7   place_name           1296 non-null   object
 8   place_url            1296 non-null   object
 9   road_address_name    1296 non-null   object
 10  x                    1296 non-null   object
 11  y                    1296 non-null   object
dtypes: object(12)
memory usage: 131.6+ KB


In [18]:
subway_word.head(3)

,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y
0,서울 종로구 숭인동 117,SW8,지하철역,"교통,수송 > 지하철,전철 > 수도권1호선",,21160608,02-6110-1271,동묘앞역 1호선,http://place.map.kakao.com/21160608,서울 종로구 종로 359,127.016364227103,37.5732166600283
1,서울 강북구 미아동 66-1,SW8,지하철역,"교통,수송 > 지하철,전철 > 수도권4호선",,21160555,02-6110-4161,미아사거리역 4호선,http://place.map.kakao.com/21160555,서울 강북구 도봉로 지하 50,127.03008663628454,37.61327836400571
2,서울 성북구 삼선동1가 14,SW8,지하철역,"교통,수송 > 지하철,전철 > 수도권4호선",,21160826,02-6110-4191,한성대입구역 4호선,http://place.map.kakao.com/21160826,서울 성북구 삼선교로 지하 1,127.00601781685579,37.58842461354086


In [19]:
subway_word = subway_word[['place_name','x','y']]

## 지하철 데이터 geojson으로 변경하기

In [20]:
# geojson변환 위해 좌표지정
from shapely.geometry import Point
pd.reset_option('display.float_format')
subway_word['x'] = subway_word.x.astype(float)
subway_word['y'] = subway_word.y.astype(float)

# geojson으로 변환
geometry = [Point(xy) for xy in zip(subway_word.x, subway_word.y)]
subway_w_gpd = gpd.GeoDataFrame(subway_word, crs='EPSG:4326', geometry=geometry)

In [21]:
subway_w_gpd.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1296 entries, 0 to 1588
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   place_name  1296 non-null   object  
 1   x           1296 non-null   float64 
 2   y           1296 non-null   float64 
 3   geometry    1296 non-null   geometry
dtypes: float64(2), geometry(1), object(1)
memory usage: 50.6+ KB


In [22]:
# 결과 저장
subway_w_gpd.to_file("서울시_지하철위치.geojson", driver='GeoJSON')

## 수도권 지하철 시각화

In [23]:
import folium

#서울시 중앙으로 오게 설정
center = (37.56, 126.97)
zoom = 12

#folium.Map으로 지도 그리기
m = folium.Map(location = center, 
               zoom_start = zoom,
               tiles = 'http://api.vworld.kr/req/wmts/1.0.0/BB730805-E115-3140-9139-BB79B7E3B74F/Base/{z}/{y}/{x}.png',
               attr = '서울시'
              )

# hilight 색 지정
purple = {'fillColor': 'purple', 'color': 'purple'}


# 시각화
folium.GeoJson(subway_w_gpd,name='seoul_bs_district', style_function=lambda x:purple).add_to(m)
m

# 상권 인근 지하철 찾기

In [24]:
# 상권 코드 & gid 데이터 
# 거리 행렬 생성을 위한 좌표 추출

bs_loc = df_gid[['x','y']]

subway_loc = subway_w_gpd[['x','y']]

In [27]:
from scipy.spatial.distance import cdist
from haversine import haversine

#cdist(행, 열)함수를 사용해 거리를 계산
# cityblock, euclidean 으로 해도 결과는 동일함을 확인했음

dist_mat = cdist(bs_loc, subway_loc, metric = haversine)

In [28]:
# close_subway_index = dist_mat.argsort()는 실제 위치에 대한 단순한 정보만을 가져온 것
# close_subway_index = dist_mat.argsort()에 [:, 0]으로 가장 첫번째(가장 거리가 짧은 = 가까운) 지하철의 index값을 불러온다.

close_subway_index1 = dist_mat.argsort()[:, 0] # axis = 0 (가장 가까운 곳)
close_subway_index2 = dist_mat.argsort()[:, 1] # axis = 1 (2번째로 가까운 곳)
close_subway_index3 = dist_mat.argsort()[:, 2] # axis = 2 (3번째로 가까운 곳)

In [29]:
# 상권 좌표 기준 가까운 지하철 TOP3 입력
df_base['가까운_지하철_1'] = subway_w_gpd.iloc[close_subway_index1]['place_name'].values 
df_base['가까운_지하철_2'] = subway_w_gpd.iloc[close_subway_index2]['place_name'].values 
df_base['가까운_지하철_3'] = subway_w_gpd.iloc[close_subway_index3]['place_name'].values 
# 새로운 시리즈를 만들 때는 list, ndarray를 사용하는 것이 바람직하다!

In [30]:
df_base.head()

,상권_코드,상권_이름,가까운_지하철_1,가까운_지하철_2,가까운_지하철_3
0,1000001,골목상권,안국역 3호선,종로3가역 5호선,종로3가역 5호선
1,1000002,골목상권,신설동역 우이신설선,신설동역 우이신설선,신설동역 우이신설선
2,1000003,골목상권,종로3가역 5호선,종로3가역 5호선,종로3가역 5호선
3,1000004,골목상권,혜화역 4호선,혜화역 4호선,종로3가역 5호선
4,1000005,골목상권,경복궁역 3호선,독립문역 3호선,광화문역 5호선


- 5번째 정도 까지 구해봤는데 가까운 지하철 2와 동일하다거나 하는 등 큰 차이가 없음
- 3번째, 4번째 지하철역을 카카오맵 상에 좌표로 찍고 상권코드에 해당하는 시장과 도보 거리 계산 해보면 보통 30분거리
- 3번째부터는 접근성이 좋다는 지표로 쓰는게 맞을까 의구심이 들었음 (물론 전부 다해본건 아니므로, 단정 지을 순 없음)

In [31]:
# df_base와 상권 geometry 데이터 결합
sub_near_bs = pd.merge(df_base, df_gid, on='상권_코드')

In [32]:
# geodataframe로 변환
sub_near_bs = gpd.GeoDataFrame(sub_near_bs)

In [33]:
# 결과 확인
sub_near_bs

,상권_코드,상권_이름_x,가까운_지하철_1,가까운_지하철_2,가까운_지하철_3,상권_이름_y,x,y,geometry
0,1000001,골목상권,안국역 3호선,종로3가역 5호선,종로3가역 5호선,골목상권,126.986400,37.582041,"MULTIPOLYGON (((126.98591 37.58549, 126.98626 ..."
1,1000002,골목상권,신설동역 우이신설선,신설동역 우이신설선,신설동역 우이신설선,골목상권,127.022589,37.573212,"MULTIPOLYGON (((127.02335 37.57240, 127.02335 ..."
2,1000003,골목상권,종로3가역 5호선,종로3가역 5호선,종로3가역 5호선,골목상권,126.988420,37.575664,"MULTIPOLYGON (((126.98909 37.57418, 126.98771 ..."
3,1000004,골목상권,혜화역 4호선,혜화역 4호선,종로3가역 5호선,골목상권,126.994996,37.588721,"MULTIPOLYGON (((126.99403 37.59000, 126.99419 ..."
4,1000005,골목상권,경복궁역 3호선,독립문역 3호선,광화문역 5호선,골목상권,126.967764,37.597109,"MULTIPOLYGON (((126.97014 37.59995, 126.97301 ..."
...,...,...,...,...,...,...,...,...,...
1485,1001486,전통시장,천호역 5호선,천호역 5호선,천호역 5호선,전통시장,127.128157,37.542089,"MULTIPOLYGON (((127.12908 37.54171, 127.12902 ..."
1486,1001487,전통시장,천호역 5호선,천호역 5호선,천호역 5호선,전통시장,127.127667,37.541139,"MULTIPOLYGON (((127.12797 37.54047, 127.12794 ..."
1487,1001488,전통시장,천호역 5호선,천호역 5호선,천호역 5호선,전통시장,127.127584,37.533863,"MULTIPOLYGON (((127.12847 37.53489, 127.12850 ..."
1488,1001489,전통시장,둔촌동역 5호선,둔촌동역 5호선,강동역 5호선,전통시장,127.135463,37.527724,"MULTIPOLYGON (((127.13638 37.52813, 127.13629 ..."


In [34]:
sub_near_bs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1490 entries, 0 to 1489
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   상권_코드      1490 non-null   object  
 1   상권_이름_x    1490 non-null   object  
 2   가까운_지하철_1  1490 non-null   object  
 3   가까운_지하철_2  1490 non-null   object  
 4   가까운_지하철_3  1490 non-null   object  
 5   상권_이름_y    1490 non-null   object  
 6   x          1490 non-null   float64 
 7   y          1490 non-null   float64 
 8   geometry   1490 non-null   geometry
dtypes: float64(2), geometry(1), object(6)
memory usage: 116.4+ KB


In [35]:
# 결과 저장
sub_near_bs.to_file('상권인근 지하철.geojson', driver='GeoJSON')